Implementation of Algorithm HSM, presented on pages 89 and 90.

In [ ]:
#Block 1: loading the packages
import numpy as np
import scipy as sc
import random
from scipy import linalg as la
from numpy.linalg import norm
import scipy.sparse as sparse
from scipy.sparse import rand as rndma
import time

In [ ]:
#Block 2: generating the rows of the starting sign-matrix 
def sparsevecgen(k):
    
    ones = np.random.choice(range(a,b))
    ones *= dim/100.0
    ones = int(np.trunc(ones))
    
    v = np.zeros(dim)
    ind = range(dim)
    random.shuffle(ind)
    ind = ind[:ones]
    v[ind] = 1
    v[k] = np.random.choice([0,-v[k]])
    
    return v

In [ ]:
#Block 4: translative power method
def pwrmthd(A):  
    
    dneg = np.diag(A)
    if (dneg < 0).any():
        h = abs(np.amin(dneg))
    else:
        h = 0
    
    A = A + (h+1)*np.identity(dim)
    v0 = np.array([1 for i in xrange(dim)])
    v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    v1 = np.round(v1,prec)
    while norm(v0-v1) > Eps*10:
        v0 = v1
        v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
        v1 = np.round(v1,prec-1)
    return v1

In [ ]:
#Block 5: computing the spectral abscissa
def leading(A):
    
    evals = np.linalg.eig(A)[0] #set of eigenvalues 
    return np.amax(np.real(evals)) #spectral abscissa

In [ ]:
#Block 6: a solution to the LP
def lp_solution(A,v,supp,tau):
    
    D = len(supp)
    X = np.copy(A)
    
    ind = np.argsort(v)[::-1] #reordering
    ind = ind[:D]
    
    for i in xrange(dim): #constructing the solution
        S = 0
        for l in ind:
            if (i != l):
                S += A[i,l]
                X[i,l] = 0
            else:
                if (X[i,i] == 1) and (S < tau - 1):
                    S += 2
                    X[i,i] = -1
                elif (X[i,i] == 1) and (S == tau - 1):
                    S += 1
                    X[i,i] = 0
                elif (X[i,i] == 0):
                    S += 1
                    X[i,i] = -1
                
            if (S == tau):
                break          
        
    return np.round(X,prec)

In [ ]:
#Block 7: implementing the the greedy method for minimization on the ball 
#of radius k
def selective_greedy(A,tau):
    
    X = np.copy(A)
    v0 = pwrmthd(X) #computing the leading eigenvalue
    supp = list(np.where(v0 != 0)[0]) #getting the support
    notsupp = list(set(range(dim)) - set(supp))
    notsupp.sort()

    while True: #constructing the solution X_k
        Z = np.copy(X)
        v = v0
        X = lp_solution(A,v,supp,tau)
        X[notsupp] = Z[notsupp]
        
        for k in supp:
            olddot = np.dot(Z[k],v)
            newdot = np.dot(X[k],v)
            olddot = np.round(olddot,prec)
            newdot = np.round(newdot,prec)
            if (olddot < newdot) or (abs(olddot - newdot) < 1e-6): #see the Appendix
                X[k] = Z[k]
        
        v0 = pwrmthd(X)
        spect_abs = np.round(leading(X),prec)
        
        '''if matrices of iterations k-1 and k match on the support, 
        OR if they have the same leading eigenvector, 
        OR if the spectral abscissa of X_k is negative
        we finish the greedy method''' 
        if (X[supp] == Z[supp]).all() or (v == v0).all() or (spect_abs < 0):
            return np.round(X,prec), spect_abs
        else:
            supp = list(np.where(v0 != 0)[0])
            notsupp = list(set(range(dim)) - set(supp))
            notsupp.sort()

In [ ]:
#Block 8: moving k forward and computing the maximal spectral abscissa
#on the corresponding ball
def forward(A,spectfin,k):
    
    while (spectfin > 0):
        
        if (spectfin > 6): 
            k += 2
        else:
            k += 1
        
        Xstar, spectfin = selective_greedy(A,k)
        
    
    return Xstar, spectfin

In [ ]:
#Block 9: moving k backward and computing the maximal spectral 
#abscissa on the corresponding ball
def backward(A,X,spect,k):
    
    while (spect <= 0):
        print spect, k
        Xstar = np.copy(X)
        spectfin = spect
        k -= 1
        X, spect = selective_greedy(A,k)
        
    
    return Xstar, spectfin

In [ ]:
#Block 10: the DFS algorithm, returning the upper triangular matrix with restored plus signs
def the_tree(A,X):
    
    minus_diag = []
    for i in xrange(dim):
        if (X[i,i] == -1):
            minus_diag.append(i)
            X[i,i] = 0
    
    tree = []
    vertices = [i for i in xrange(dim)]
    ind = [i for i in xrange(dim)]
    Aredux_col = np.copy(A)
    Xredux_row = np.copy(X)
    Xredux_col = np.copy(X)
    
    while (vertices != []):
    
        #finding the source(s)
        sources = [j for j in vertices if (np.sum(Xredux_row[:,j]) == 0)]
        indy = np.argmax([np.sum((Aredux_col - Xredux_col)[j]) for j in sources])
        
        the_source = sources[indy]
        
        #taking theem out
        tree.append(the_source)
        vertices.remove(the_source)
        Aredux_col = A[np.ix_(ind,vertices)]
        Xredux_col = A[np.ix_(ind,vertices)]
        Xredux_row = X[np.ix_(vertices,ind)]
       
    for x in xrange(len(tree)):
        i = tree.pop(0)
        for j in tree:
            X[i,j] = A[i,j]
            
    for i in minus_diag:
        X[i,i] = -1

    return X

In [ ]:
#Block 11: finding the closest Hurwitz sign-stable matrix (HSM)
def closest_sign_stable(A):
    

    row_sums = [np.sum(A[i]) for i in xrange(dim)] 

    k0 = np.amax(row_sums)
    k0 = np.trunc(k0/2)
    k1 = np.amin(row_sums)
    seg = k0 + 1
    
    '''doing a bisection in k until we obtain a matrix with
    appropriate spectral abscissa (Step X)'''

    while (seg >= 1):
        

        Xstar, spect_abs = selective_greedy(A,k0)
        

        
        if (spect_abs > 3):
            k1 = k0
            seg /= 2.0
            seg = np.ceil(seg)
            k0 += seg

        elif (spect_abs == 0):
            k1 = k0
            seg /= 2.0
            seg = np.ceil(seg)
            k0 -= seg

        else:
            k1 = k0
            break

    if (spect_abs > 0):
        '''if a last obtained matrix has a spectral abscissa bigger than zero
        we move k forward untill we obtain the optimal solution'''
        Xstar, spect_radius = forward(A,spect_abs,k1)
    else:
        '''if a last obtained matrix has a non-negative spectral abscissa, 
        we move k backwards untill we get a minimal k for which
        we have the closest sign-stable matrix'''
        Xstar, spect_radius = backward(A,Xstar,spect_abs,k1)


    
    '''running the DFS algorithm (if we don't have cycles in the final matrix) 
    and restoring the + signs'''
    Z = np.copy(Xstar)
    for i in xrange(dim):
        Z[i,i] = 0
    if (leading(Z) == 0):
        Z = np.copy(Xstar)
        Xstar = the_tree(A,Z) 

    
    
    
    return Xstar, norm(Xstar - A,np.Inf) 

In [ ]:
#Block 11: running the algorithm HSM
prec = 9 #setting the rounding parameter
Eps = 10**(-prec)
dim = 10 #setting the dimension
(a,b) = (66,95) #setting the density of the starting matrix
A = np.array([sparsevecgen(k) for k in xrange(dim)], dtype = float) #generating start. matrix
K = closest_sign_stable(A) #running the algorithm 
print K